In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns = ["EIN", "NAME"])

application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Determine the number of unique values in each column.

application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type_counts[app_type_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"

classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts[classification_counts > 1].value_counts()

,count
count,
2,5
6,3
3,3
9,2
32,2
5,2
15,2
10,2
17326,1


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = classification_counts[classification_counts < 1800].index.tolist()

# Replace in dataframe

for cls in classifications_to_replace:
  application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls,"Other")

# Check to make sure replacement was successful
application_df["CLASSIFICATION"].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`

optimized_model_df = application_df.copy()
optimized_model_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Split our preprocessed data into our features and target arrays

y = optimized_model_df["IS_SUCCESSFUL"]

X = optimized_model_df.drop(columns = "IS_SUCCESSFUL")

In [ ]:
X = pd.get_dummies(X)

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
#Check shape of scaled training data.

X_train_scaled.shape

(25724, 43)

In [ ]:
# Define input for model.
# [0] represents the number of rows while (25,724) [1] represents the number of columns OR the features for our model (43).

input_dim = X_train_scaled.shape[1]

In [ ]:
# Optimization - Create a method that creates a new Sequential model with hyperparameter options.

def create_optimized_model(hp):
  optimized_model = tf.keras.models.Sequential()

  activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])

#First hidden layer.

  optimized_model.add(tf.keras.layers.Dense(units = hp.Int("units_",
                                                    min_value = 1,
                                                    max_value = 10,
                                                    step = 2),
                                                    activation = activation,
                                            input_dim = X_train_scaled.shape[1]

  ))

#Second hidden layer.

  optimized_model.add(tf.keras.layers.Dense(units = hp.Int("units_2",
                                                    min_value = 16,
                                                    max_value = 64,
                                                    step = 8),
                                                    activation = activation
  ))

#Create a Dropuout layer.

  optimized_model.add(tf.keras.layers.Dropout(rate = hp.Choice(
                                                    "dropout_rate",
                                                    [0.1, 0.2, 0.3, 0.4, 0.5])
  ))

#Output layer.

  optimized_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

#Compile model.

  optimized_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

  return optimized_model


In [ ]:
# Import the keras tuner library.

import keras_tuner as kt

# Set up the tuner.

tuner = kt.Hyperband(
    create_optimized_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Run the kerastuner search for best hyperparameters.

tuner.search(X_train_scaled, y_train, epochs = 20, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.7325947284698486

Best val_accuracy So Far: 0.7341107726097107
Total elapsed time: 00h 29m 51s


In [22]:
# Get the best model hyperparameters.

best_model_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_model_hyperparameters.values

{'activation': 'sigmoid',
 'units_': 5,
 'units_2': 16,
 'dropout_rate': 0.5,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [23]:
# Evaluate best model against full test data.

best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7341 - loss: 0.5604
Loss: 0.5603592991828918, Accuracy: 0.7341107726097107


In [24]:
# Save the model in .h5 and .keras

best_model.save("AlphabetSoupCharity_Optimization.h5")

best_model.save("AlphabetSoupCharity_Optimization.keras")